<a href="https://colab.research.google.com/github/Master-Shuvam/MultiTrack-Cricket/blob/main/Copy_of_23f3002495_UPDATED_PLAYER_TRACKING_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

WELCOME TO THE HIGHLY STABILISED UPDATED PLAYER ID TRACKING SYSTEM

✅STEPS TO RUN IT SMOOTHLY:
1. Open the notebook in Google Colab
2. Run all the cells at once
3. Mount google drive for direct importing option from drive
4. When prompted just enter the video file datapath
5. Then just enjoy !!!!!!!! everything is automatic
6. Automatic downloading will be done







ALL DEPENDENCIES WILL BE DOWLOADED HERE

In [ ]:
"""
Run this cell first to install all required packages
"""

!pip install ultralytics -q
!pip install supervision -q
!pip install opencv-python-headless -q
!pip install torch torchvision -q
!pip install scipy -q

print(" All dependencies installed successfully!")

MOUNT GOOGLE DRIVE DIRECTLY IN COLAB

In [ ]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from collections import defaultdict, deque
from pathlib import Path
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

print("✓ Libraries imported and Drive mounted successfully!")
print(f"✓ Using device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

MY SPECIALLY DESIGNED CUSTOM ENGINEERED SPATIAL ANCHORED MULTI-FEATURE TRACKER FOR LONG TERM ID STABILITY

In [ ]:
"""
SPATIAL ANCHORED PLAYER TRACKER
================================
Key Innovation: Players CANNOT teleport!
- Enforces spatial continuity
- Maximum 12 active players (11 players + extras)
- ID created ONLY if far from ALL existing players
- Strong temporal memory (20 frames)
- Multi-feature matching (position, size, color, speed, direction)
"""

import cv2
import numpy as np
from collections import deque
from scipy.optimize import linear_sum_assignment

class SpatialAnchoredTracker:

    def __init__(self, max_players=12, spatial_threshold=300, memory_frames=30):
        """
        Args:
            max_players: Maximum players on field (11 players + buffer)
            spatial_threshold: Minimum distance for new ID (pixels)
            memory_frames: Number of frames to remember per track
        """
        self.max_players = max_players
        self.spatial_threshold = spatial_threshold  # New ID only if > 300px away
        self.memory_frames = memory_frames

        self.next_id = 1
        self.active_tracks = {}  # Currently visible players
        self.memory_bank = {}    # Recently lost players (keep for recovery)
        self.frame_count = 0

        # ID STABILITY ENHANCEMENT
        self.track_confidence = {}  # Track how stable each ID is
        self.last_match_quality = {}  # Quality of last match for each track

        print(f"✓ Spatial Anchored Tracker initialized:")
        print(f"  - Max Players: {max_players}")
        print(f"  - Spatial Threshold: {spatial_threshold}px")
        print(f"  - Memory Frames: {memory_frames}")

    def _extract_features(self, frame, box):
        x1, y1, x2, y2 = map(int, box)

        h, w = frame.shape[:2]
        x1 = max(0, min(x1, w-1))
        y1 = max(0, min(y1, h-1))
        x2 = max(0, min(x2, w-1))
        y2 = max(0, min(y2, h-1))

        if x2 <= x1 or y2 <= y1:
            return None

        try:
            roi = frame[y1:y2, x1:x2]
            if roi.size == 0:
                return None

            # Multi-space color histograms
            roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            hist_h = cv2.calcHist([roi_hsv], [0], None, [30], [0, 180])
            hist_s = cv2.calcHist([roi_hsv], [1], None, [32], [0, 256])

            hist_h = cv2.normalize(hist_h, hist_h).flatten()
            hist_s = cv2.normalize(hist_s, hist_s).flatten()

            features = np.concatenate([hist_h, hist_s])
            return features
        except:
            return None

    def _compare_features(self, feat1, feat2):

        if feat1 is None or feat2 is None:
            return 0.0

        similarity = np.corrcoef(feat1, feat2)[0, 1]
        if np.isnan(similarity):
            return 0.0

        return max(0, min(1, (similarity + 1) / 2))

    def _get_center(self, box):

        return np.array([(box[0] + box[2]) / 2, (box[1] + box[3]) / 2])

    def _get_size(self, box):

        return (box[2] - box[0]) * (box[3] - box[1])

    def _calculate_distance(self, box1, box2):

        c1 = self._get_center(box1)
        c2 = self._get_center(box2)
        return np.linalg.norm(c1 - c2)

    def _calculate_iou(self, box1, box2):

        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])

        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = self._get_size(box1)
        area2 = self._get_size(box2)
        union = area1 + area2 - intersection

        return intersection / union if union > 0 else 0.0

    def _predict_next_position(self, track):

        positions = track['position_history']

        if len(positions) < 2:
            return track['box']

        # Use last 5 positions for prediction
        recent = list(positions)[-5:]
        centers = np.array([self._get_center(box) for box in recent])

        if len(centers) >= 3:
            # Fit motion model
            velocities = np.diff(centers, axis=0)
            avg_velocity = np.mean(velocities[-3:], axis=0)

            # Predict with damping
            last_center = centers[-1]
            predicted_center = last_center + avg_velocity * 1.2

            # Build predicted box
            last_box = recent[-1]
            w, h = last_box[2] - last_box[0], last_box[3] - last_box[1]

            predicted_box = np.array([
                predicted_center[0] - w/2,
                predicted_center[1] - h/2,
                predicted_center[0] + w/2,
                predicted_center[1] + h/2
            ])

            return predicted_box

        return np.array(track['box'])

    def _calculate_match_score(self, detection, track, det_feature):
        """
        Calculate comprehensive match score with ID STABILITY BOOST

        Considers:
        - Spatial distance (position continuity)
        - IoU (overlap)
        - Appearance (color similarity)
        - Size consistency
        - Speed consistency
        - Direction consistency
        - TRACK CONFIDENCE (new - rewards stable tracks)
        """
        det_box = detection[:4]
        pred_box = self._predict_next_position(track)

        # 1. Spatial distance (MOST IMPORTANT - no teleportation!)
        distance = self._calculate_distance(det_box, pred_box)

        # ADAPTIVE distance threshold based on track stability
        track_id = None
        for tid, t in self.active_tracks.items():
            if t == track:
                track_id = tid
                break


        confidence = self.track_confidence.get(track_id, 0.5)
        max_expected_distance = 150 + (confidence * 150)  # 150-300px based on stability

        spatial_score = max(0, 1 - distance / max_expected_distance)

        # 2. IoU
        iou = self._calculate_iou(det_box, pred_box)

        # 3. Appearance
        appearance_score = self._compare_features(det_feature, track['appearance'])

        # 4. Size consistency
        det_size = self._get_size(det_box)
        track_size = np.mean([self._get_size(b) for b in list(track['position_history'])[-5:]])
        size_ratio = min(det_size, track_size) / max(det_size, track_size, 1)

        # 5. Speed consistency
        speed_score = 1.0
        if len(track['position_history']) >= 2:
            recent_speeds = []
            positions = list(track['position_history'])[-5:]
            for i in range(1, len(positions)):
                dist = self._calculate_distance(positions[i], positions[i-1])
                recent_speeds.append(dist)

            if recent_speeds:
                avg_speed = np.mean(recent_speeds)
                current_speed = distance

                if avg_speed > 0:
                    speed_ratio = min(current_speed, avg_speed) / max(current_speed, avg_speed)
                    speed_score = speed_ratio

        # 6. Direction consistency
        direction_score = 1.0
        if len(track['position_history']) >= 3:
            positions = list(track['position_history'])[-5:]
            centers = np.array([self._get_center(box) for box in positions])

            if len(centers) >= 2:
                prev_direction = centers[-1] - centers[-2]
                det_center = self._get_center(det_box)
                pred_center = self._get_center(pred_box)
                current_direction = det_center - pred_center

                if np.linalg.norm(prev_direction) > 0 and np.linalg.norm(current_direction) > 0:
                    # Cosine similarity
                    cos_sim = np.dot(prev_direction, current_direction) / (
                        np.linalg.norm(prev_direction) * np.linalg.norm(current_direction)
                    )
                    direction_score = (cos_sim + 1) / 2


        stability_bonus = confidence * 0.3  # Up to 30% boost!

        # 8. Recent match quality - if last match was good, boost this match
        last_quality = self.last_match_quality.get(track_id, 0.5)
        quality_bonus = last_quality * 0.1  # Up to 10% boost

        # Weighted combination - HEAVILY FAVOR STABLE TRACKS
        base_score = (
            0.35 * spatial_score +      # Position continuity
            0.25 * appearance_score +   # Visual similarity
            0.15 * iou +                # Overlap
            0.10 * size_ratio +         # Size consistency
            0.08 * speed_score +        # Speed consistency
            0.07 * direction_score      # Direction consistency
        )

        # Add stability bonuses
        final_score = base_score + stability_bonus + quality_bonus

        return final_score, distance

    def _is_far_from_all_tracks(self, detection_box, threshold):
        """
        Check if detection is far from ALL existing tracks
        This prevents creating new IDs for existing players
        """
        if not self.active_tracks:
            return True

        det_center = self._get_center(detection_box)

        for track in self.active_tracks.values():
            track_box = track['box']
            if isinstance(track_box, list):
                track_box = np.array(track_box)
            track_center = self._get_center(track_box)
            distance = np.linalg.norm(det_center - track_center)

            if distance < threshold:
                return False  # Too close to existing player

        return True  # Far from all players

    def update(self, detections, frame=None):
        """
        Update tracker with spatial anchoring

        Strategy:
        1. Match detections to existing tracks (strict matching)
        2. Try to recover from memory bank
        3. Only create new ID if:
           - Far from ALL existing players (> threshold)
           - AND total players < max_players
        """
        self.frame_count += 1

        # Extract features
        det_features = []
        if frame is not None:
            for det in detections:
                feat = self._extract_features(frame, det[:4])
                det_features.append(feat)
        else:
            det_features = [None] * len(detections)


        for track in self.active_tracks.values():
            track['predicted_box'] = self._predict_next_position(track)

        # === PHASE 1: MATCH TO ACTIVE TRACKS ===
        # Use GREEDY MATCHING instead of Hungarian for stability
        # Strategy: Match high-confidence tracks FIRST (they get priority)

        matched_pairs = []
        unmatched_detections = list(range(len(detections)))
        unmatched_tracks = list(self.active_tracks.keys())

        if len(detections) > 0 and len(self.active_tracks) > 0:
            track_ids = list(self.active_tracks.keys())
            track_confidences = [self.track_confidence.get(tid, 0.3) for tid in track_ids]
            sorted_indices = np.argsort(track_confidences)[::-1]  # Descending order
            sorted_track_ids = [track_ids[i] for i in sorted_indices]

            # GREEDY MATCHING: Each stable track claims its best detection
            for track_id in sorted_track_ids:
                if track_id not in unmatched_tracks:
                    continue

                track = self.active_tracks[track_id]
                confidence = self.track_confidence.get(track_id, 0.3)

                best_det_idx = None
                best_score = -999
                best_distance = 999

                # Find best detection for this track
                for det_idx in unmatched_detections:
                    det = detections[det_idx]
                    score, distance = self._calculate_match_score(det, track, det_features[det_idx])

                    if score > best_score:
                        best_score = score
                        best_distance = distance
                        best_det_idx = det_idx


                if best_det_idx is not None:

                    score_threshold = 0.25 - (confidence * 0.15)  # 0.25 down to 0.10
                    distance_threshold = 200 + (confidence * 150)   # 200 up to 350px


                    if confidence > 0.8:
                        score_threshold = 0.05  # Almost always accept
                        distance_threshold = 400  # Very large distance OK

                    if best_score > score_threshold and best_distance < distance_threshold:
                        matched_pairs.append((best_det_idx, track_id))
                        unmatched_detections.remove(best_det_idx)
                        unmatched_tracks.remove(track_id)


                        self.last_match_quality[track_id] = best_score


        for det_idx, track_id in matched_pairs:
            det = detections[det_idx]
            box = np.array(det[:4])

            track = self.active_tracks[track_id]

            # Adaptive smoothing based on movement speed
            prev_box = track['box']
            if isinstance(prev_box, list):
                prev_box = np.array(prev_box)
            else:
                prev_box = np.array(prev_box)

            movement = np.linalg.norm(self._get_center(box) - self._get_center(prev_box))

            if movement > 80:  # Fast movement
                smoothed_box = 0.3 * prev_box + 0.7 * box
            else:  # Slow movement
                smoothed_box = 0.7 * prev_box + 0.3 * box

            track['box'] = smoothed_box.tolist()
            track['position_history'].append(smoothed_box)
            track['confidence'] = det[4] if len(det) > 4 else 1.0
            track['frames_tracked'] += 1
            track['frames_lost'] = 0

            # Update appearance with retention
            if det_features[det_idx] is not None:
                if track['appearance'] is not None:
                    track['appearance'] = 0.85 * track['appearance'] + 0.15 * det_features[det_idx]
                else:
                    track['appearance'] = det_features[det_idx]

            # Limit history
            if len(track['position_history']) > self.memory_frames:
                track['position_history'].popleft()

        # Age unmatched active tracks
        for track_id in unmatched_tracks:
            self.active_tracks[track_id]['frames_lost'] += 1

            # Get current confidence
            current_confidence = self.track_confidence.get(track_id, 0.3)

            if current_confidence > 0.8:
                confidence_decay = 0.01  # Very slow decay for stable tracks
                max_frames_lost = 80     # Wait longer before removing
            elif current_confidence > 0.6:
                confidence_decay = 0.03
                max_frames_lost = 60
            else:
                confidence_decay = 0.05
                max_frames_lost = 40

            # Decrease confidence when track is lost
            self.track_confidence[track_id] = max(0.3, current_confidence - confidence_decay)

            # Move to memory bank based on confidence-adjusted threshold
            if self.active_tracks[track_id]['frames_lost'] > max_frames_lost:
                self.memory_bank[track_id] = self.active_tracks[track_id]
                self.memory_bank[track_id]['lost_at_frame'] = self.frame_count
                del self.active_tracks[track_id]

        # === PHASE 2: RECOVERY FROM MEMORY BANK ===
        for det_idx in unmatched_detections[:]:
            det = detections[det_idx]
            det_box = np.array(det[:4])
            det_feat = det_features[det_idx]

            best_match_id = None
            best_score = 0.25  # Lower threshold for recovery

            for mem_id, mem_track in self.memory_bank.items():
                frames_since_lost = self.frame_count - mem_track['lost_at_frame']

                # Only recover recent losses (< 3 seconds)
                if frames_since_lost > 150:
                    continue

                # Calculate match score
                mem_box = mem_track['box']
                if isinstance(mem_box, list):
                    mem_box = np.array(mem_box)

                distance = self._calculate_distance(det_box, mem_box)
                appearance_sim = self._compare_features(det_feat, mem_track['appearance'])

                # Recovery focuses on appearance and reasonable distance
                score = 0.6 * appearance_sim + 0.4 * max(0, 1 - distance / 400)

                if score > best_score:
                    best_score = score
                    best_match_id = mem_id

            # Recover track
            if best_match_id is not None:
                self.active_tracks[best_match_id] = self.memory_bank[best_match_id]
                self.active_tracks[best_match_id]['box'] = det_box.tolist()
                self.active_tracks[best_match_id]['frames_lost'] = 0
                self.active_tracks[best_match_id]['position_history'].append(det_box)

                # Restore confidence (but reduced)
                if best_match_id in self.track_confidence:
                    self.track_confidence[best_match_id] = max(0.4, self.track_confidence[best_match_id] - 0.1)
                else:
                    self.track_confidence[best_match_id] = 0.4

                if det_feat is not None:
                    if self.active_tracks[best_match_id]['appearance'] is not None:
                        self.active_tracks[best_match_id]['appearance'] = (
                            0.7 * self.active_tracks[best_match_id]['appearance'] + 0.3 * det_feat
                        )
                    else:
                        self.active_tracks[best_match_id]['appearance'] = det_feat

                del self.memory_bank[best_match_id]
                unmatched_detections.remove(det_idx)

        # === PHASE 3: CREATE NEW IDs (STRICT RULES) ===
        for det_idx in unmatched_detections:
            det = detections[det_idx]
            det_box = np.array(det[:4])

            # RULE 1: Check if we're at max capacity
            if len(self.active_tracks) >= self.max_players:
                continue  # Don't create new ID

            # RULE 2: Must be far from ALL existing players
            if not self._is_far_from_all_tracks(det_box, self.spatial_threshold):
                continue  # Too close to existing player

            # RULE 3: Check memory bank too
            too_close_to_memory = False
            for mem_track in self.memory_bank.values():
                mem_box = mem_track['box']
                if isinstance(mem_box, list):
                    mem_box = np.array(mem_box)
                distance = self._calculate_distance(det_box, mem_box)
                if distance < self.spatial_threshold * 0.7:  # Even stricter for memory
                    too_close_to_memory = True
                    break

            if too_close_to_memory:
                continue

            # All checks passed - create new track
            track_id = self.next_id
            self.next_id += 1

            self.active_tracks[track_id] = {
                'box': det_box.tolist(),
                'predicted_box': det_box.tolist(),
                'position_history': deque([det_box], maxlen=self.memory_frames),
                'appearance': det_features[det_idx],
                'confidence': det[4] if len(det) > 4 else 1.0,
                'frames_tracked': 1,
                'frames_lost': 0
            }

            # Initialize new track with LOW confidence (must prove itself)
            self.track_confidence[track_id] = 0.3
            self.last_match_quality[track_id] = 0.5

        # Clean old memory
        to_delete = []
        for mem_id, mem_track in self.memory_bank.items():
            if self.frame_count - mem_track['lost_at_frame'] > 200:  # 4 seconds
                to_delete.append(mem_id)
        for mem_id in to_delete:
            del self.memory_bank[mem_id]

        # Return results
        results = []
        for track_id, track in self.active_tracks.items():
            box = track['box']
            results.append([box[0], box[1], box[2], box[3], track_id])

        return results

print("✓ Spatial Anchored Tracker defined!")




COMPLETE END TO END PIPELINE

In [ ]:
class CricketPlayerTrackingPipeline:
    def __init__(self, model_name='yolov8x.pt'):
        print(f"Loading YOLO model: {model_name}...")
        from ultralytics import YOLO
        self.model = YOLO(model_name)

        # Spatial Anchored Tracker - prevents ID explosion
        self.tracker = SpatialAnchoredTracker(
            max_players=12,           # Max 12 players on field
            spatial_threshold=300,    # 300px minimum for new ID
            memory_frames=20          # Remember 20 frames
        )
        print("✓ Model and Spatial Anchored Tracker initialized!")

    def detect_players(self, frame):
        """Detect players using YOLO"""
        results = self.model(frame, classes=[0], verbose=False)[0]

        detections = []
        if len(results.boxes) > 0:
            boxes = results.boxes.xyxy.cpu().numpy()
            confidences = results.boxes.conf.cpu().numpy()

            for box, conf in zip(boxes, confidences):
                if conf > 0.45:  # Balanced threshold
                    detections.append([
                        float(box[0]), float(box[1]),
                        float(box[2]), float(box[3]),
                        float(conf)
                    ])

        return detections

    def draw_tracking_results(self, frame, tracked_objects):
        """Draw tracking results with stable IDs"""
        annotated_frame = frame.copy()

        colors = [
            (0, 255, 0), (255, 0, 0), (0, 0, 255), (255, 255, 0),
            (255, 0, 255), (0, 255, 255), (128, 0, 128), (255, 128, 0),
            (0, 128, 255), (128, 255, 0), (255, 0, 128), (200, 100, 50)
        ]

        for obj in tracked_objects:
            x1, y1, x2, y2, track_id = obj
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            color = colors[int(track_id) % len(colors)]

            # Thick box
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 4)

            # Large ID label
            label = f"Player {int(track_id)}"
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1.2
            thickness = 3

            (text_width, text_height), _ = cv2.getTextSize(label, font, font_scale, thickness)

            # Background
            cv2.rectangle(
                annotated_frame,
                (x1, y1 - text_height - 20),
                (x1 + text_width + 20, y1),
                color,
                -1
            )

            # Text
            cv2.putText(
                annotated_frame,
                label,
                (x1 + 10, y1 - 10),
                font,
                font_scale,
                (255, 255, 255),
                thickness
            )

            # Center point
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            cv2.circle(annotated_frame, (center_x, center_y), 8, color, -1)

        return annotated_frame

    def process_video(self, input_path, output_path, skip_frames=1):
        """Process video with spatial anchored tracking"""
        print(f"\n{'='*60}")
        print(f"🏏 SPATIAL ANCHORED CRICKET TRACKING")
        print(f"{'='*60}\n")

        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            raise ValueError(f"Could not open video: {input_path}")

        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        print(f"Video: {width}x{height} @ {fps}fps")
        print(f"Total: {total_frames} frames ({total_frames/fps:.1f}s)")
        print(f"Processing every {skip_frames} frame(s)\n")

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        frame_count = 0
        last_tracked = []

        # Reset tracker
        self.tracker = SpatialAnchoredTracker(
            max_players=12,
            spatial_threshold=300,
            memory_frames=20
        )

        max_id_seen = 0

        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                frame_count += 1

                if frame_count % skip_frames == 0:
                    detections = self.detect_players(frame)
                    tracked_objects = self.tracker.update(detections, frame=frame)

                    # Track max ID
                    if tracked_objects:
                        current_max = max(int(obj[4]) for obj in tracked_objects)
                        max_id_seen = max(max_id_seen, current_max)

                    annotated_frame = self.draw_tracking_results(frame, tracked_objects)
                    last_tracked = tracked_objects

                    if frame_count % 50 == 0:
                        progress = (frame_count / total_frames) * 100
                        active = len(self.tracker.active_tracks)
                        memory = len(self.tracker.memory_bank)

                        # Show confidence levels of active tracks
                        high_conf = sum(1 for tid in self.tracker.active_tracks.keys()
                                       if self.tracker.track_confidence.get(tid, 0) > 0.8)

                        print(f"⏳ {progress:.1f}% | Frame {frame_count}/{total_frames} | "
                              f"Active: {active} (Stable: {high_conf}) | Memory: {memory} | Max ID: {max_id_seen}")
                else:
                    annotated_frame = self.draw_tracking_results(frame, last_tracked)

                out.write(annotated_frame)

        finally:
            cap.release()
            out.release()

        print(f"\n{'='*60}")
        print(f"✅ COMPLETE!")
        print(f"{'='*60}")
        print(f"Max ID reached: {max_id_seen}")
        print(f"Output: {output_path}")
        print(f"{'='*60}\n")

print("✓ Spatial Anchored Pipeline defined!")

USER INPUT AND RUNNING THE PIPELINE. WHEN PROMPTED JUST ENTER THE FILE PATH

In [ ]:
import os
from pathlib import Path

print("="*60)
print(" SPATIAL ANCHORED CRICKET TRACKING")
print("="*60)
print()

video_path = input("Enter video path: ").strip()

if os.path.isdir(video_path):
    print(f"\n⚠️  Folder detected!")
    try:
        files = os.listdir(video_path)
        video_files = [f for f in files if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]

        if video_files:
            print("\nAvailable videos:")
            for i, vf in enumerate(video_files, 1):
                print(f"  {i}. {vf}")

            choice = input(f"\nSelect (1-{len(video_files)}): ").strip()
            idx = int(choice) - 1
            video_path = os.path.join(video_path, video_files[idx])
    except:
        print("Error reading folder")
        raise SystemExit

if not os.path.isfile(video_path):
    print(f"\n❌ File not found: {video_path}")
    raise SystemExit

print(f"\n✅ Video: {video_path}")

video_name = Path(video_path).stem
output_path = f"/content/output_{video_name}_spatial_anchored.mp4"

skip_input = input("\nFrame skip (1-3, default=1): ").strip()
skip_frames = int(skip_input) if skip_input.isdigit() else 1

print("\n Processing...\n")

try:
    pipeline = CricketPlayerTrackingPipeline(model_name='yolov8x.pt')
    pipeline.process_video(video_path, output_path, skip_frames=skip_frames)

    print("\n SUCCESS!")
    print(f"Output: {output_path}")

except Exception as e:
    print(f"\n Error: {str(e)}")

AUTOMATIC DOWNLOADING

In [ ]:
# ============================================================================
# CELL 6: DOWNLOAD THE OUTPUT FILES
# ============================================================================

from google.colab import files
import glob

print("📥 Downloading output...\n")

try:
    output_files = glob.glob("/content/output_*_spatial_anchored.mp4")

    if output_files:
        latest = max(output_files, key=os.path.getctime)
        size_mb = os.path.getsize(latest) / (1024*1024)

        print(f"File: {os.path.basename(latest)} ({size_mb:.1f} MB)")
        print("Starting download...\n")

        files.download(latest)
        print("✅ Download started!")
    else:
        print(" No output files found")

except Exception as e:
    print(f" Error: {str(e)}")

THANK YOU FOR USING MY NOTEBOOK. PLEASE GIVE YOUR VALUABLE FEEDBACK.